# DB에서 상장 기업 정보를 받아 네이버 증권에서 주가 수집하기

In [8]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
import pymysql
import time

In [12]:
pymysql.install_as_MySQLdb()
load_dotenv(dotenv_path=".env_db")

False

In [4]:
# mysql에서 테이블 불러오기
engine = create_engine(f"{os.getenv('db')}+{os.getenv('dbtype')}://{os.getenv('id')}:{os.getenv('pw')}@{os.getenv('host')}/{os.getenv('database')}")
conn = engine.connect()
data = pd.read_sql('2024_07_29_stock_company_info', con=conn)
conn.close()

NoSuchModuleError: Can't load plugin: sqlalchemy.dialects:None.None

In [ ]:
def str2int(x):
    x = int(x.replace(",", ""))
    return x

In [79]:
stock_price_detail = {}

for idx, code in enumerate(data['종목코드']):
    url = f"https://finance.naver.com/item/main.naver?code={code}" 
    r = requests.get(url)
    print(r.status_code, f"{idx+1}/{len(data['종목코드'])} 수집중", end="\r")
    soup = bs(r.text, 'lxml')
    
    
    
    # 가격
    price = int((soup.select_one(".today").text).strip("\n").split("\n")[1].replace(",", ""))
    # 변동금액
    price_chage = int((soup.select_one(".today").text).strip("\n").split("\n")[9].replace(",", ""))
    # 변화율
    rate_of_chage = float(((soup.select_one(".today").text).strip("\n").split("\n")[13]+(soup.select_one(".today").text).strip("\n").split("\n")[15]).replace("%",""))
    stock_price_detail.setdefault('현재가', []).append(price)
    stock_price_detail.setdefault('변동금액', []).append(price_chage)
    stock_price_detail.setdefault('변화율', []).append(rate_of_chage)
    
    
    
    table = soup.select_one(".no_info")
    for tr in table.select("tr"):
        for td in tr.select('td'):
            stock_price_detail.setdefault(td.select_one('span').text, []).append(str2int(td.select_one("span.blind").text))
    time.sleep(5)
    
    
stock_price_detail

KeyboardInterrupt: 

In [84]:
data

,증권종류,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,유가증권,산일전기,062040,"전동기, 발전기 및 전기 변환 · 공급 · 제어 장치 제조업","유입, 몰드, 주상, 건식 변압기 등",2024-07-29,12월,박동석,http://www.sanil.co.kr,경기도
1,유가증권,에이치에스효성,487570,기타 금융업,지주사업,2024-07-29,12월,조현상..,http://www.hshyosung.com,서울특별시
2,"코스닥, 투자주의종목",엔에이치스팩31호,481890,금융 지원 서비스업,금융지원서비스업,2024-07-26,12월,이시형,,서울특별시
3,코스닥,SK증권제13호스팩,473950,금융 지원 서비스업,기업인수목적 주식회사,2024-07-25,12월,임율표,,서울특별시
4,코스닥,엑셀세라퓨틱스,373110,기초 의약물질 제조업,CellCor SFD/CD(세포배양배지),2024-07-15,12월,이의일,,서울특별시
...,...,...,...,...,...,...,...,...,...,...
2700,"유가증권, KTOP30, KOSPI200, KRX300",유한양행,000100,의약품 제조업,"의약품(삐콤씨, 안티푸라민, 렉라자, 로수바미브, 코푸시럽 등), 생활용품(유한락스...",1962-11-01,12월,대표이..,http://www.yuhan.co.kr,서울특별시
2701,"유가증권, KOSPI200, KRX300",CJ대한통운,000120,도로 화물 운송업,"Contract Logistics, 포워딩, 항만하역, 해운, 택배국제특송, SCM...",1956-07-02,12월,신영수..,http://www.cjlogistics.com,서울특별시
2702,유가증권,경방,000050,종합 소매업,"섬유류(면사,면혼방사,면직물,면혼방직물,화섬사,화섬직물) 제조,도매,수출입",1956-03-03,12월,"김준,..",http://www.kyungbang.co.kr,서울특별시
2703,유가증권,유수홀딩스,000700,회사 본부 및 경영 컨설팅 서비스업,지주사업,1956-03-03,12월,송영규,http://www.eusu-holdings.com,서울특별시


# 수집과 동시에 DB에 저장하기 + 예외처리 하기

In [81]:

print(today)

2024_07_29


In [82]:
# DB에 접속해서 저장하는 함수
def stock_info_to_db(idx, code, df):
    from sqlalchemy import create_engine
    import pymysql
    from dotenv import load_dotenv
    from datetime import date
    pymysql.install_as_MySQLdb()
    load_dotenv(dotenv_path=".env_db")
    today = str(date.today()).replace("-","_")
    
    engine = create_engine(f"{os.getenv('db')}+{os.getenv('dbtype')}://{os.getenv('id')}:{os.getenv('pw')}@{os.getenv('host')}/{os.getenv('database')}")
    conn = engine.connect()
    df.to_sql(f"{today}_stock_price_info", con=conn, if_exists='append', index=False)
    conn.close()
    
    return print(f"{today}, {idx}, {code}, {'저장완료':<30s}", end="\r")
    

In [85]:
errors = {}
for idx, (company, code) in enumerate(zip(data['회사명'], data['종목코드'])):
    stock_price_detail = {}
    url = f"https://finance.naver.com/item/main.naver?code={code}" 
    try:
        r = requests.get(url)
        print(r.status_code, f"{idx+1}/{len(data['종목코드'])} 수집중                    ", end="\r")
        soup = bs(r.text, 'lxml')
        # 가격
        price = int((soup.select_one(".today").text).strip("\n").split("\n")[1].replace(",", ""))
        # 변동금액
        price_chage = int((soup.select_one(".today").text).strip("\n").split("\n")[9].replace(",", ""))
        # 변화율
        rate_of_chage = float(((soup.select_one(".today").text).strip("\n").split("\n")[13]+(soup.select_one(".today").text).strip("\n").split("\n")[15]).replace("%",""))
        stock_price_detail.setdefault('회사명', []).append(company)
        stock_price_detail.setdefault('종목코드', []).append(code)
        stock_price_detail.setdefault('현재가', []).append(price)
        stock_price_detail.setdefault('변동금액', []).append(price_chage)
        stock_price_detail.setdefault('변화율', []).append(rate_of_chage)
        table = soup.select_one(".no_info")
        for tr in table.select("tr"):
            for td in tr.select('td'):
                stock_price_detail.setdefault(td.select_one('span').text, []).append(str2int(td.select_one("span.blind").text))
        df = pd.DataFrame(stock_price_detail)
        stock_info_to_db(idx, code, df)
        time.sleep(5)
    except Exception as e:
        print(e)
        print(r.status_code, f"{idx+1}/{len(data['종목코드'])} 수집중 에러", end="\r")
        errors.setdefault("에러", []).append(code)  # 에러난 코드들 모음
        
stock_price_detail

KeyboardInterrupt: 